# Melt basics

In [1]:
import MagmaPandas as mp

Import some melt data from a csv file

In [2]:
melt_file = "./data/melts.csv"

melt = mp.read_melt(melt_file, index_col=["name"])
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,
PI032-04-01,49.402603,14.539062,6.509044,10.058735,9.757536,3.737374,0.697769,0.132497,2.548581,0.289823,0.628502,1.440208,0.078857,0.145452,0.033958,100
PI032-04-02,48.832207,15.116896,6.508618,9.688642,9.927146,3.564377,0.940308,0.135905,2.652552,0.336327,0.672345,1.330865,0.083879,0.165394,0.044540,100
PI041-02-02,49.124916,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.463920,0.655494,0.047654,0.068020,0.021157,100
PI041-03-01,45.941086,15.753453,6.909844,10.996510,10.778833,3.346468,1.144852,0.112117,3.153303,0.551992,0.797715,0.293341,0.079499,0.086863,0.054123,100
PI041-03-03,45.313339,15.909531,6.831676,11.175276,10.966834,3.417548,1.167550,0.088899,3.262463,0.517891,0.821116,0.311131,0.081713,0.079495,0.055540,100


On top of the regular MagmaFrame methods, Melt has additional methods for calculating

- NBO/T,
- melt densities,
- liquidus temperatures,
- Fe3+/Fe2+ ratios,
- melt FeO and Fe2O3,
- olivine-melt Fe-Mg partitioning coefficients and
- H2O-CO2 saturation pressures.

Let's check the current configuration of MagmaPandas ...

In [3]:
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
ΔfO2........................................1
Melt Fe3+/Fe2+........................Sun2024
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_15
Volatile solubility model......IaconoMarziano
Volatile species........................mixed
#############################################



... and calculate liquidus temperatures with a melt-only thermometer at 4 kbar. The configuration is currently set to use equation 15 from Putirka (2008).

In [4]:
P_bar = 4e3

temperatures = melt.temperature(P_bar=P_bar)
temperatures

name
PI032-04-01    1446.506126
PI032-04-02    1448.470282
PI041-02-02    1413.913988
PI041-03-01    1471.475055
PI041-03-03    1470.435383
PI041-05-04    1393.411880
PI041-05-06    1418.544001
PI041-07-01    1461.241836
PI041-07-02    1464.626056
PI052-01-02    1383.362277
Name: T_K, dtype: float64

We can switch to a different thermometer by changing the configuration:

In [5]:
mp.configuration.melt_thermometer = "putirka2008_16"

temperatures_new = melt.temperature(P_bar=P_bar)
temperatures_new

name
PI032-04-01    1441.689105
PI032-04-02    1438.624306
PI041-02-02    1395.139631
PI041-03-01    1442.485008
PI041-03-03    1441.510218
PI041-05-04    1370.343048
PI041-05-06    1399.939176
PI041-07-01    1440.598610
PI041-07-02    1439.236317
PI052-01-02    1368.655862
Name: T_K, dtype: float64

With these temperatures we can calculate melt Fe3+/Fe2+ ratios. The configuration is currently set to fO2 buffered at QFM+1 and the Borisov et al. (2018) Fe3+/Fe2+ model.

In [6]:
melt.head()

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total
name,,,,,,,,,,,,,,,,
PI032-04-01,49.402603,14.539062,6.509044,10.058735,9.757536,3.737374,0.697769,0.132497,2.548581,0.289823,0.628502,1.440208,0.078857,0.145452,0.033958,100
PI032-04-02,48.832207,15.116896,6.508618,9.688642,9.927146,3.564377,0.940308,0.135905,2.652552,0.336327,0.672345,1.330865,0.083879,0.165394,0.044540,100
PI041-02-02,49.124916,16.950138,4.861451,9.183852,10.238358,3.788617,1.077791,0.153258,2.804343,0.561031,0.463920,0.655494,0.047654,0.068020,0.021157,100
PI041-03-01,45.941086,15.753453,6.909844,10.996510,10.778833,3.346468,1.144852,0.112117,3.153303,0.551992,0.797715,0.293341,0.079499,0.086863,0.054123,100
PI041-03-03,45.313339,15.909531,6.831676,11.175276,10.966834,3.417548,1.167550,0.088899,3.262463,0.517891,0.821116,0.311131,0.081713,0.079495,0.055540,100


In [7]:
print(mp.configuration)
mp.configuration.dfO2 = 2


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
ΔfO2........................................1
Melt Fe3+/Fe2+........................Sun2024
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_16
Volatile solubility model......IaconoMarziano
Volatile species........................mixed
#############################################



In [8]:
moles = melt.moles()

In [9]:
moles.FeO_Fe2O3_calc(0.33, wtpc=False)

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,total,Fe2O3
name,,,,,,,,,,,,,,,,,
PI032-04-01,0.498648,0.086478,0.097943,0.108783,0.061930,0.036570,0.004492,0.001133,0.019353,0.001238,0.008661,0.048484,0.002517,0.002751,0.000581,0.989782,0.010218
PI032-04-02,0.496051,0.090491,0.098564,0.105452,0.063410,0.035101,0.006093,0.001169,0.020272,0.001446,0.009325,0.045090,0.002695,0.003148,0.000767,0.989537,0.010463
PI041-02-02,0.518555,0.105437,0.076501,0.103871,0.067958,0.038769,0.007257,0.001370,0.022270,0.002507,0.006686,0.023077,0.001591,0.001345,0.000378,0.988787,0.011213
PI041-03-01,0.480702,0.097135,0.107784,0.123283,0.070919,0.033945,0.007641,0.000994,0.024822,0.002445,0.011396,0.010237,0.002631,0.001703,0.000960,0.988298,0.011702
PI041-03-03,0.474557,0.098185,0.106660,0.125399,0.072220,0.034697,0.007800,0.000789,0.025705,0.002296,0.011741,0.010868,0.002706,0.001560,0.000986,0.988084,0.011916
PI041-05-04,0.512278,0.117337,0.061949,0.107859,0.064936,0.047763,0.010978,0.001253,0.020040,0.003742,0.007460,0.016494,0.002973,0.002438,0.001071,0.989286,0.010714
PI041-05-06,0.492420,0.106249,0.075050,0.101574,0.077597,0.041217,0.009602,0.001508,0.029047,0.002786,0.009285,0.020756,0.003609,0.002551,0.001141,0.987197,0.012803
PI041-07-01,0.473225,0.094020,0.103617,0.108615,0.074906,0.031789,0.008501,0.001267,0.027836,0.002460,0.006689,0.035661,0.002488,0.003207,0.001000,0.987641,0.012359
PI041-07-02,0.479135,0.096393,0.103903,0.113359,0.075639,0.032479,0.009113,0.001195,0.027930,0.002739,0.005337,0.021588,0.002444,0.002763,0.001021,0.987519,0.012481


In [10]:
mp.configuration.Fe3Fe2_model = "borisov"
Fe3Fe2_QFM1 = melt.Fe3Fe2(T_K=temperatures, P_bar=P_bar)
Fe3Fe2_QFM1

name
PI032-04-01    0.501960
PI032-04-02    0.497872
PI041-02-02    0.455355
PI041-03-01    0.507279
PI041-03-03    0.517809
PI041-05-04    0.477988
PI041-05-06    0.492887
PI041-07-01    0.512346
PI041-07-02    0.504972
PI052-01-02    0.511342
dtype: float64

Changing the fO2 buffer to lower log offset values reduces the amount of Fe3+

In [11]:
mp.configuration.dfO2 = 0

melt.Fe3Fe2(T_K=temperatures, P_bar=P_bar)

name
PI032-04-01    0.193495
PI032-04-02    0.191919
PI041-02-02    0.175530
PI041-03-01    0.195545
PI041-03-03    0.199604
PI041-05-04    0.184254
PI041-05-06    0.189997
PI041-07-01    0.197498
PI041-07-02    0.194656
PI052-01-02    0.197111
dtype: float64

With our Fe3+/Fe2+ ratios we can calculate melt FeO and Fe2O3 contents

In [12]:
melt.FeO_Fe2O3_calc(Fe3Fe2=Fe3Fe2_QFM1)

,SiO2,Al2O3,MgO,CaO,FeO,Na2O,K2O,MnO,TiO2,P2O5,CO2,H2O,F,S,Cl,Fe2O3,total
name,,,,,,,,,,,,,,,,,
PI032-04-01,49.223872,14.486461,6.485495,10.022344,6.473030,3.723852,0.695245,0.132018,2.539361,0.288775,0.626228,1.434998,0.078572,0.144926,0.033835,3.610989,100.0
PI032-04-02,48.653453,15.061559,6.484793,9.653176,6.603239,3.551329,0.936866,0.135407,2.642842,0.335096,0.669884,1.325994,0.083572,0.164789,0.044377,3.653623,100.0
PI041-02-02,48.950318,16.889894,4.844172,9.151211,7.009950,3.775152,1.073961,0.152713,2.794376,0.559037,0.462271,0.653164,0.047484,0.067778,0.021082,3.547435,100.0
PI041-03-01,45.756266,15.690078,6.882046,10.952271,7.122418,3.333006,1.140246,0.111666,3.140617,0.549772,0.794506,0.292161,0.079179,0.086514,0.053905,4.015349,100.0
PI041-03-03,45.125352,15.843528,6.803334,11.128914,7.195461,3.403370,1.162706,0.088530,3.248928,0.515743,0.817709,0.309840,0.081374,0.079165,0.055309,4.140735,100.0
PI041-05-04,47.550926,18.482696,3.857219,9.344093,6.485652,4.573285,1.597563,0.137275,2.472596,0.820543,0.507178,0.459036,0.087262,0.120753,0.058686,3.445239,100.0
PI041-05-06,46.169509,16.905232,4.720217,8.888523,7.750329,3.986434,1.411383,0.166960,3.620062,0.617108,0.637611,0.583491,0.107004,0.127627,0.063132,4.245378,100.0
PI041-07-01,45.661478,15.395014,6.706579,9.781336,7.600286,3.164057,1.285957,0.144360,3.570197,0.560783,0.472711,1.031699,0.075901,0.165164,0.056926,4.327553,100.0
PI041-07-02,45.674476,15.593199,6.644080,10.085505,7.619363,3.193726,1.361936,0.134466,3.539037,0.616858,0.372665,0.617027,0.073670,0.140577,0.057439,4.275978,100.0


Our melts also contain H2O and CO2, so let's calculate volatile saturation pressures. In the configuration we see that we're currently using the Iacono-Marziano et al. (2012) model with mixed CO2-H2O fluids:

In [13]:
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
ΔfO2........................................0
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_16
Volatile solubility model......IaconoMarziano
Volatile species........................mixed
#############################################



In [14]:
pressures_IM2012 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_IM2012

Saturation pressure...    |█████████████████████████| 100% [10/10] in 1.0s (5.59


name
PI032-04-01    5859.560466
PI032-04-02    6071.786862
PI041-02-02    4791.149912
PI041-03-01    6461.985990
PI041-03-03    6339.842577
PI041-05-04    4118.352036
PI041-05-06    5170.724099
PI041-07-01    4504.675250
PI041-07-02    3819.965205
PI052-01-02    2590.928144
dtype: float64

Lets change the volatile solubility model to Allison et al. (2022):

In [15]:
mp.configuration.volatile_solubility = 'Allison2022'
print(mp.configuration)


################ MagmaPandas ################
#############################################
General settings_____________________________
fO2 buffer................................QFM
ΔfO2........................................0
Melt Fe3+/Fe2+........................borisov
Kd Fe-Mg ol-melt.......................toplis
Melt thermometer...............putirka2008_16
Volatile solubility model.........Allison2022
Volatile species........................mixed
#############################################



In [16]:
pressures_A22 = melt.volatile_saturation_pressure(T_K=temperatures)
pressures_A22

Saturation pressure...    |█████████████████████████| 100% [10/10] in 0.6s (15.7


name
PI032-04-01    4762.967014
PI032-04-02    5382.216127
PI041-02-02    4898.391782
PI041-03-01    5839.864575
PI041-03-03    6016.042165
PI041-05-04    5258.315488
PI041-05-06    5827.032917
PI041-07-01    4519.918367
PI041-07-02    3925.172944
PI052-01-02    2597.606572
dtype: float64